In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j python-dotenv langchain-neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')


NEO4J_URI = NEO4J_URI
NEO4J_USERNAME = NEO4J_USERNAME
NEO4J_PASSWORD = NEO4J_PASSWORD
NVIDIA_API_KEY =NVIDIA_API_KEY

In [3]:
import os


os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=str(NEO4J_URI),
    username= str(NEO4J_USERNAME),
    password=str(NEO4J_PASSWORD),
)
graph

<ipython-input-4-6cc53d2d5161>:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [6]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.7 MB/s eta 0:00:00


In [73]:
query = """
// Create Knowledge node
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create and connect PYS
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create and connect GITA
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)
"""


In [74]:
graph.query(query)

[]

In [75]:

query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/Gita_questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the GITA label and connect it to Knowledge
MERGE (gita:GITA {name: 'GITA'})
MERGE (knowledge)-[:HAS_SOURCE]->(gita)

// Create and relate Chapter nodes
MERGE (chapter:GITAChapter {number: toInteger(row.chapter)})
MERGE (gita)-[:HAS_CHAPTER]->(chapter)

// Create and relate Verse nodes
MERGE (verse:GITAVerse {number: toInteger(row.verse), chapterNumber: toInteger(row.chapter)})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Relate Sanskrit text to Verse
MERGE (shloka:GITASanskrit {text: row.sanskrit})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create Translation node and relate it to Sanskrit Shloka and Verse
MERGE (translation:GITATranslation {text: row.translation})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)
MERGE (verse)-[:HAS_TRANSLATION]->(translation)

// Create Question node and ensure proper relationships
MERGE (question:GITAQuestion {text: row.question})
MERGE (question)-[:ABOUT_VERSE]->(verse)
MERGE (question)-[:ABOUT_TRANSLATION]->(translation)
MERGE (question)-[:ABOUT_SHLOKA]->(shloka)



"""


In [77]:
graph.query(query)

[]

In [78]:
query="""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/parth1609/RAGveda/main/dataset/PYS_English_Questions.csv' AS row
WITH row
WHERE row.chapter IS NOT NULL AND row.chapter <> ''
  AND row.verse IS NOT NULL AND row.verse <> ''

// Ensure the Knowledge node exists
MERGE (knowledge:Knowledge {name: 'Knowledge'})

// Create the PYS label and connect it to Knowledge
MERGE (pys:PYS {name: 'PYS'})
MERGE (knowledge)-[:HAS_SOURCE]->(pys)

// Create Chapter node and relate it to PYS
MERGE (chapter:PYSChapter {number: toInteger(row.chapter)})
MERGE (pys)-[:HAS_CHAPTER]->(chapter)

// Create Verse node and relate it to Chapter
MERGE (verse:PYSVerse {number: toInteger(row.verse), chapterNumber: toInteger(row.chapter)})
MERGE (chapter)-[:HAS_VERSE]->(verse)

// Relate Sanskrit text to Verse
MERGE (shloka:PYSSanskrit {text: row.sanskrit})
MERGE (verse)-[:HAS_SHLOKA]->(shloka)

// Create Translation node and relate it to Sanskrit Shloka and Verse
MERGE (translation:PYSTranslation {text: row.translation})
MERGE (shloka)-[:HAS_TRANSLATION]->(translation)
MERGE (verse)-[:HAS_TRANSLATION]->(translation)

// Create Question node and ensure proper relationships
MERGE (question:PYSQuestion {text: row.question})
MERGE (question)-[:ABOUT_VERSE]->(verse)
MERGE (question)-[:ABOUT_TRANSLATION]->(translation)
MERGE (question)-[:ABOUT_SHLOKA]->(shloka)


"""

In [79]:
graph.query(query)

[]

In [80]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Knowledge {name: STRING}
PYS {name: STRING}
GITA {name: STRING}
GITAChapter {number: INTEGER}
GITAVerse {number: INTEGER, chapterNumber: INTEGER}
GITASanskrit {text: STRING}
GITATranslation {text: STRING}
GITAQuestion {text: STRING}
PYSChapter {number: INTEGER}
PYSVerse {number: INTEGER, chapterNumber: INTEGER}
PYSSanskrit {text: STRING}
PYSTranslation {text: STRING}
PYSQuestion {text: STRING}
Relationship properties:

The relationships:
(:Knowledge)-[:HAS_SOURCE]->(:PYS)
(:Knowledge)-[:HAS_SOURCE]->(:GITA)
(:PYS)-[:HAS_CHAPTER]->(:PYSChapter)
(:GITA)-[:HAS_CHAPTER]->(:GITAChapter)
(:GITAChapter)-[:HAS_VERSE]->(:GITAVerse)
(:GITAVerse)-[:HAS_SHLOKA]->(:GITASanskrit)
(:GITAVerse)-[:HAS_TRANSLATION]->(:GITATranslation)
(:GITASanskrit)-[:HAS_TRANSLATION]->(:GITATranslation)
(:GITAQuestion)-[:ABOUT_VERSE]->(:GITAVerse)
(:GITAQuestion)-[:ABOUT_TRANSLATION]->(:GITATranslation)
(:GITAQuestion)-[:ABOUT_SHLOKA]->(:GITASanskrit)
(:PYSChapter)-[:HAS_VERSE]->(:PYSVerse)
(:PYSVerse

In [81]:
pip install -U --quiet langchain-nvidia-ai-endpoints

In [82]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
  model="meta/llama-3.1-70b-instruct",
  api_key=NVIDIA_API_KEY,

)


In [83]:
%pip install -qU langchain-groq

In [84]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True)



In [94]:
question = "Who was the king from Kashi who fought in the Mahabharata war?"
sanskritShloka = "सर्वस्य चाहं हृदि सन्निविष्टो मत्तः स्मृतिर्ज्ञानमपोहनं च| वेदैश्च सर्वैरहमेव वेद्यो वेदान्तकृद्वेदविदेव चाहम् || 15.15 || "
speaker = "भगवान"

try:
    response = chain.invoke({"query": f"I have this '{question}'. please give about shloka from 'Gita' or 'Pys' particular chapter , along with its structure as[shloka,tranlation], or state that no relevant information exists`"})
    print(response)  # Or display response in a suitable way for your notebook
except Exception as e:
    print(f"Error: {e}") # printing the error that has occurred
    print(f"No relevant information found for the question: {question}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (q:GITAQuestion {text: "Who was the king from Kashi who fought in the Mahabharata war?"})-[:ABOUT_VERSE]->(v:GITAVerse)-[:HAS_SHLOKA]->(s:GITASanskrit)-[:HAS_TRANSLATION]->(t:GITATranslation)
OPTIONAL MATCH (q)-[:ABOUT_SHLOKA]->(s2:GITASanskrit)-[:HAS_TRANSLATION]->(t2:GITATranslation)
RETURN [[s.text, t.text]] as result
UNION
MATCH (q:PYSQuestion {text: "Who was the king from Kashi who fought in the Mahabharata war?"})-[:ABOUT_VERSE]->(v:PYSVerse)-[:HAS_SHLOKA]->(s:PYSSanskrit)-[:HAS_TRANSLATION]->(t:PYSTranslation)
OPTIONAL MATCH (q)-[:ABOUT_SHLOKA]->(s2:PYSSanskrit)-[:HAS_TRANSLATION]->(t2:PYSTranslation)
RETURN [[s.text, t.text]] as result

Full Context:
[{'result': [['धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्| पुरुजित्कुन्तिभोजश्च शैब्यश्च नरपुङ्गवः  || 1.5 || ', 'Dhrishtaketu, Chekitana, the valiant king of Kasi, Purujit, Kuntibhoja, and Saibya—the best of men.']]}]

> Finished chain.
{'query': "I have this 'Who